In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pickle
import cmocean
import json
import f90nml
from collections import OrderedDict

import warnings
warnings.filterwarnings('ignore')
from IPython.display import Markdown, display

%matplotlib inline

In [2]:
# prep and load dictionary to save stats in
if os.path.isfile('vET-HC1905-DFO-NutChlPhys-stats.json'):
    with open('vET-HC1905-DFO-NutChlPhys-stats.json', 'r') as fstat:
        statsDict = json.load(fstat);
else:
    print('Error: No stats file')

In [3]:
statsDict['2007']['NO3']['all']

{'N': 1060,
 'Bias': -0.6905845237509283,
 'RMSE': 3.515278044470087,
 'WSS': 0.89470388601169}

In [4]:
statsDict.keys()

dict_keys(['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'])

In [5]:
varlist=('NO3', 'dSi', 'Chl', 'Chl log10', 'Temperature', 'Salinity', 'Density')
sublist=('all',)#'z < 15 m')
yearlist=[str(ii) for ii in range(2007,2019)]
statsDict['2007'].keys()
width=6
prec=3
trVar={'NO3': (r'NO\ :sub:`3` (\ :math:`{\mu}`\ M )',(' ')),
       'dSi': (r'Dissolved Silica (\ :math:`{\mu}`\ M )',(' ')),
       'Chl': (r' Chl a (µg L\ :sup:`-1`\ )',(' ')),
       'Chl log10': (r'log10[Chl a (µg L\ :sup:`-1`\ )]',(' ')),
       'Temperature': (r'Conservative',('Temperature (°C)')),
       'Salinity': (r'Reference',('Salinity (g kg\ :sup:`-1`\ )')),
       'Density': (r' Density (kg m\ :sup:`-3`\ )',(' '))}

In [6]:
def makeTable2():
    lines=list()
    lines.append(r'+----------------------------------------+-------+--------+--------+--------+------+'+'\n')
    lines.append(r'|                               Variable | Year  | Bias   | RMSE   | WSS    |   N  |'+'\n')
    for var in varlist:
        var1=True
        var2=False
        for sub in sublist:
            for year in yearlist:
                if var1:
                    vartxt=r'| '+'{:>38}'.format(trVar[var][0]) 
                    varsep=r'+----------------------------------------'
                elif var2:
                    vartxt=r'| '+'{:>38}'.format(trVar[var][1])
                    varsep='+                                        '
                else: 
                    vartxt=r'| '+'{:38}'.format(' ')
                    varsep='+                                        '
                yeartxt=r' | '+'{:5}'.format(str(year))
                st=statsDict[year][var][sub]
                statlist=('{:{w}.{p}f}'.format(st['Bias'], w=width, p=prec),
                          '{:{w}.{p}f}'.format(st['RMSE'], w=width, p=prec),
                          '{:{w}.{p}f}'.format(st['WSS'], w=width, p=prec),
                          '{:4d}'.format(st['N']))
                sepline=(varsep+'+-------+--------+--------+--------+------+'+'\n')
                lines.append(sepline)
                lines.append(vartxt+yeartxt+' | '+' | '.join(statlist)+' |\n')
                var2=True if var1 else False
                var1=False

    lines.append(r'+----------------------------------------+-------+--------+--------+--------+------+'+'\n')
    for el in lines:
        print(el)
    return lines

In [7]:
if len(sublist)>1:
    lines=makeTable1()
elif len(sublist)==1:
    lines=makeTable2()
else:
    print('Error, sublist:',sublist)

+----------------------------------------+-------+--------+--------+--------+------+

|                               Variable | Year  | Bias   | RMSE   | WSS    |   N  |

+----------------------------------------+-------+--------+--------+--------+------+

|     NO\ :sub:`3` (\ :math:`{\mu}`\ M ) | 2007  | -0.691 |  3.515 |  0.895 | 1060 |

+                                        +-------+--------+--------+--------+------+

|                                        | 2008  | -0.572 |  4.201 |  0.904 | 1096 |

+                                        +-------+--------+--------+--------+------+

|                                        | 2009  |  0.371 |  3.711 |  0.915 | 1097 |

+                                        +-------+--------+--------+--------+------+

|                                        | 2010  | -0.391 |  3.599 |  0.940 | 1461 |

+                                        +-------+--------+--------+--------+------+

|                                        | 2011  |  0.

In [8]:
def makeTable1():
    lines=list()
    lines.append(r'+----------------------------------------+----------+-------+--------+--------+--------+------+'+'\n')
    lines.append(r'|                               Variable | Subset   | Year  | Bias   | RMSE   | WSS    |   N  |'+'\n')
    #lines.append(r'+--------------------------------------------+------------+----------+---------+---------+---------+------+'+'\n')
    #sepline=r'+--------------------------------------------+------------+----------+---------+---------+---------+------+'+'\n'
    for var in varlist:
        var1=True
        var2=False
        for sub in sublist:
            sub1=True
            for year in yearlist:
                if var1:
                    vartxt=r'| '+'{:>38}'.format(trVar[var][0]) 
                    varsep=r'+----------------------------------------'
                elif var2:
                    vartxt=r'| '+'{:>38}'.format(trVar[var][1])
                    varsep='+                                        '
                else: 
                    vartxt=r'| '+'{:38}'.format(' ')
                    varsep='+                                        '
                subtxt=r' | '+'{:8}'.format(sub) if sub1 else r' | '+'{:8}'.format(' ')
                subsep=r'+----------' if sub1 else '+          '
                yeartxt=r' | '+'{:5}'.format(str(year))
                st=statsDict[year][var][sub]
                statlist=('{:{w}.{p}f}'.format(st['Bias'], w=width, p=prec),
                          '{:{w}.{p}f}'.format(st['RMSE'], w=width, p=prec),
                          '{:{w}.{p}f}'.format(st['WSS'], w=width, p=prec),
                          '{:4d}'.format(st['N']))
                sepline=(varsep+subsep+'+-------+--------+--------+--------+------+'+'\n')
                lines.append(sepline)
                lines.append(vartxt+subtxt+yeartxt+' | '+' | '.join(statlist)+' |\n')
                var2=True if var1 else False
                var1=False
                sub1=False
    lines.append(r'+----------------------------------------+----------+-------+--------+--------+--------+------+'+'\n')
    for el in lines:
        print(el)
    return lines

In [9]:
f = open('/data/eolson/results/MEOPAR/docs/NEMO36ModelEvaluation201905/HC1905statsTBL.txt', 'w')
f.writelines(lines)
f.close()